In [14]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content="Dog are great companions",
        metadata ={"source":"mammal-pets-docs"}
    ),
    Document(
        page_content="Cats are independent creatures.",
        metadata={"source": "mammal-pets-docs"}
    ),
    Document(
        page_content="Python is a high-level programming language.",
        metadata={"source": "programming-languages-docs"}
    ),
    Document(
        page_content="The Eiffel Tower is a wrought iron lattice tower on the Champ de Mars in Paris.",
        metadata={"source": "world-landmarks-docs"}
    ),
    Document(
        page_content="Quantum computing is an area of computer science that uses quantum-mechanical phenomena to perform computation.",
        metadata={"source": "technology-docs"}
    )
]

In [15]:
# VectorStores
from langchain_chroma import Chroma

In [16]:
import os
from dotenv import load_dotenv
from langchain_groq import ChatGroq
load_dotenv()
groq_api_key=os.getenv("GROQ_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

llm=ChatGroq(groq_api_key=groq_api_key, model="Llama3-8b-8192")
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x7f1e5017a060>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x7f1e5017a960>, model_name='Llama3-8b-8192', groq_api_key=SecretStr('**********'))

In [17]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

In [18]:
# VectorStores
from langchain_chroma import Chroma
Vectorstores=Chroma.from_documents(documents,embedding=embeddings)
Vectorstores

In [21]:
# Async query
await Vectorstores.asimilarity_search("cat")

[Document(metadata={'source': 'mammal-pets-docs'}, page_content='Cats are independent creatures.'),
 Document(metadata={'source': 'mammal-pets-docs'}, page_content='Cats are independent creatures.'),
 Document(metadata={'source': 'mammal-pets-docs'}, page_content='Cats are independent creatures.'),
 Document(metadata={'source': 'mammal-pets-docs'}, page_content='Dog are great companions')]

In [ ]:
Vectorstores.similarity_search_with_score("cat")

In [26]:
# Retrivers
from typing import List

from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriver=RunnableLambda(Vectorstores.similarity_search).bind(k=1)
retriver.batch(["cat","dog"])
retriver.invoke("cat")

[Document(metadata={'source': 'mammal-pets-docs'}, page_content='Cats are independent creatures.')]

In [29]:
Vectorstores.as_retriever(
    search_type="similarity",
    search_kwargs={"k":1}
)
retriver.batch(["cat","dog"])

[[Document(metadata={'source': 'mammal-pets-docs'}, page_content='Cats are independent creatures.')],
 [Document(metadata={'source': 'mammal-pets-docs'}, page_content='Dog are great companions')]]

In [30]:
"RAG"

from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message = """
Answer this question using the provided context only.

{question}

Context:
{context}
"""

prompt = ChatPromptTemplate.from_messages([('human',message)])
rag_chain = {"context" : retriver,"question":RunnablePassthrough()}|prompt|llm
response = rag_chain.invoke("tell me about dogs")

print(response.content)


According to the provided context, dogs are great companions.


In [1]:
from ollama import Ollama

   

ollama = Ollama(model="phi3")  # Replace with your model name
ollama.start()

ImportError: cannot import name 'Ollama' from 'ollama' (/workspace/.pyenv_mirror/user/current/lib/python3.12/site-packages/ollama/__init__.py)